## Imports

In [2]:
import numpy as np
import pandas as pd
import datetime
import math
import random

pd.options.display.max_columns = 50

In [3]:
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler

## Functions

In [28]:
#------------------#
# BUILD OUR MODEL  #
#------------------#

def implicit_als(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):
    """ 
    Function
    --------
        Implementation of Alternating Least Squares with implicit data. We iteratively
        compute the user (x_u) and item (y_i) vectors using the following formulas:

        x_u = ((Y.T*Y + Y.T*(Cu - I) * Y) + lambda*I)^-1 * (X.T * Cu * p(u))
        y_i = ((X.T*X + X.T*(Ci - I) * X) + lambda*I)^-1 * (Y.T * Ci * p(i))
 
    Parameters
    ----------
        sparse_data (csr_matrix): Our sparse user-by-item matrix
 
        alpha_val (int): The rate in which we'll increase our confidence
        in a preference with more interactions.
 
        iterations (int): How many times we alternate between fixing and 
        updating our user and item vectors
 
        lambda_val (float): Regularization value
 
        features (int): How many latent features we want to compute.
    
    Returns:
    --------
        X (csr_matrix): user vectors of size users-by-features
        
        Y (csr_matrix): item vectors of size items-by-features
     """

    # Calculate the confidence for each value in our data
    confidence = sparse_data * alpha_val
    
    # Get the size of user rows and item columns
    user_size, item_size = sparse_data.shape
    
    # We create the user vectors X of size users-by-features, the item vectors
    # Y of size items-by-features and randomly assign the values.
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))
    
    #Precompute I and lambda * I
    X_I = sparse.eye(user_size)
    Y_I = sparse.eye(item_size)
    
    I = sparse.eye(features)
    lI = lambda_val * I

    # Start main loop. For each iteration we first compute X and then Y
    for i in range(iterations):
        print ('iteration %d of %d' % (i+1, iterations))
        
        # Precompute Y-transpose-Y and X-transpose-X
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)

        # Loop through all users
        for u in range(user_size):

            # Get the user row.
            u_row = confidence[u,:].toarray() 

            # Calculate the binary preference p(u)
            p_u = u_row.copy()
            p_u[p_u != 0] = 1.0

            # Calculate Cu and Cu - I
            CuI = sparse.diags(u_row, [0])
            Cu = CuI + Y_I

            # Put it all together and compute the final formula
            yT_CuI_y = Y.T.dot(CuI).dot(Y)
            yT_Cu_pu = Y.T.dot(Cu).dot(p_u.T)
            X[u] = spsolve(yTy + yT_CuI_y + lI, yT_Cu_pu)

        for i in range(item_size):

            # Get the item column and transpose it.
            i_row = confidence[:,i].T.toarray()

            # Calculate the binary preference p(i)
            p_i = i_row.copy()
            p_i[p_i != 0] = 1.0

            # Calculate Ci and Ci - I
            CiI = sparse.diags(i_row, [0])
            Ci = CiI + X_I

            # Put it all together and compute the final formula
            xT_CiI_x = X.T.dot(CiI).dot(X)
            xT_Ci_pi = X.T.dot(Ci).dot(p_i.T)
            Y[i] = spsolve(xTx + xT_CiI_x + lI, xT_Ci_pi)

    return X, Y
    

In [158]:
#------------------------------#
# CREATE USER RECOMMENDATIONS  #
#------------------------------#

def recommend(user_id, data_sparse, user_vecs, item_vecs, item_lookup, top_3_from_train, num_items=3):
    """
    Function:
    --------
        Recommend items for a given user given a trained model
    
    Parameters:
    ----------
        user_id (int): The id of the user we want to create recommendations for.
        
        data_sparse (csr_matrix): Our original training data.
        
        user_vecs (csr_matrix): The trained user x features vectors
        
        item_vecs (csr_matrix): The trained item x features vectors
        
        item_lookup (pandas.DataFrame): Used to map charity_id to charity names
        
        num_items (int): How many recommendations we want to return:
        
    Returns:
    -------
        recommendations (pandas.DataFrame): DataFrame with num_items charity names and scores
    
    """
    
    # Get all interactions by the user
    # Return Top 3 from Training Data if user not in DataBase
    try: user_interactions = data_sparse[user_id,:].toarray()
    except: return top_3_from_train
    
#     # Get all interactions by the user
#     user_interactions = data_sparse[user_id,:].toarray()

    # We don't want to recommend items the user has consumed. So let's
    # set them all to 0 and the unknowns to 1.
    user_interactions = user_interactions.reshape(-1) + 1 #Reshape to turn into 1D array
    user_interactions[user_interactions > 1] = 0

    # This is where we calculate the recommendation by taking the 
    # dot-product of the user vectors with the item vectors.
    rec_vector = user_vecs[user_id,:].dot(item_vecs.T).toarray()

    # Let's scale our scores between 0 and 1 to make it all easier to interpret.
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = user_interactions*rec_vector_scaled
   
    # Get all the charity indices in order of recommendations (descending) and
    # select only the top "num_items" items. 
    item_idx = np.argsort(recommend_vector)[::-1][:num_items]

    charities = []
    scores = []

    # Loop through our recommended charity indicies and look up the actual charity name
    for idx in item_idx:
        charities.append(item_lookup['Advised Charity'].loc[item_lookup.charity_id == str(idx)].iloc[0])
        scores.append(recommend_vector[idx])

    # Create a new dataframe with recommended charity names and scores
    recommendations = pd.DataFrame({'charity': charities, 'score': scores})
    
    return recommendations

In [50]:
def recommend_similar_charities(item_lookup,item_vec,item_name="",item_id=-1,num_sim_char=10):
    '''
    '''
    if item_id == -1:
        # Charity Name --> Charity_id
        item_id = int(item_lookup['charity_id'].loc[item_lookup['Advised Charity'] == str(item_name)].iloc[0])

    # Get the item row for selected charity.
    item_vec = item_vecs[item_id].T

    # Calculate the similarity score between selected charity and other charities
    # and select the top 10 most similar.
    scores = item_vecs.dot(item_vec).toarray().reshape(1,-1)[0]
    top_10 = np.argsort(scores)[::-1][:10]

    charities = []
    charity_scores = []

    print("Charities Similar to:", item_lookup['Advised Charity'].loc[item_lookup.charity_id == str(item_id)].iloc[0],"\n")

    # Get and print the actual charity names and scores
    for idx in top_10:
        charities.append(item_lookup['Advised Charity'].loc[item_lookup.charity_id == str(idx)].iloc[0])
        charity_scores.append(scores[idx])

    similar = pd.DataFrame({'charities': charities, 'score': charity_scores})

    print (similar)
    return similar

In [124]:
def create_collab_filtering_model(df):
    '''
    '''
    raw_data = df[['User ID','Advised Charity','Amount']]
    # Drop Rows with Missing Values
    data = raw_data.dropna()

    # Convert charity names into numerical IDs
    data['user_id'] = data['User ID'].astype("category").cat.codes
    data['charity_id'] = data['Advised Charity'].astype("category").cat.codes

    # Create a lookup frame so we can get the charity names back in 
    # readable form later.
    user_lookup = data[['user_id', 'User ID']].drop_duplicates()
    user_lookup['user_id'] = user_lookup['user_id'].astype(str)
    
    # Create a lookup frame so we can get the charity names back in 
    # readable form later.
    item_lookup = data[['charity_id', 'Advised Charity']].drop_duplicates()
    item_lookup['charity_id'] = item_lookup['charity_id'].astype(str)
    
    data = data.drop(['User ID', 'Advised Charity'], axis=1)

    # Drop Rows with $0 Donation
    data = data.loc[data.Amount > 0]
    
    # Create lists of all users, charities and amounts
    users = list(np.sort(data['user_id'].unique()))
    charities = list(np.sort(data['charity_id'].unique()))
    amounts = list(data.Amount)

    # Get the rows and columns for our new matrix
    rows = data['user_id'].astype(int)
    cols = data['charity_id'].astype(int)

    # Contruct a sparse matrix for our users and items containing amounts
    data_sparse = sparse.csr_matrix((amounts, (rows, cols)), shape=(len(users), len(charities)))
    
    # Training The Model 
    # (Creating User-Latent Vector and Item-Latent Vector)
    user_vecs, item_vecs = implicit_als(data_sparse, iterations=20, features=20, alpha_val=40)
    
    return user_vecs, item_vecs, data_sparse, item_lookup, user_lookup

## Loading Data

In [83]:
user_and_charity_df = pd.read_csv('../data/user_and_charity_df.csv',parse_dates=True)
user_and_charity_df['Date'] = user_and_charity_df['Date'].apply(lambda x: pd.to_datetime(x))

## Implicit Collaborative Filtering

In [19]:
raw_data = user_and_charity_df[['User ID','Advised Charity','Amount']]

In [20]:
# Drop Rows with Missing Values
data = raw_data.dropna()

# Convert charity names into numerical IDs
data['user_id'] = data['User ID'].astype("category").cat.codes
data['charity_id'] = data['Advised Charity'].astype("category").cat.codes

# Create a lookup frame so we can get the charity names back in 
# readable form later.
item_lookup = data[['charity_id', 'Advised Charity']].drop_duplicates()
item_lookup['charity_id'] = item_lookup['charity_id'].astype(str)

In [21]:
item_lookup.head()

,charity_id,Advised Charity
0,439,Room to Grow National Inc
3,557,Wildlife Conservation Society
14,396,Play Soccer to Give Corp
85,280,Lemon Bay Junior Golf Foundation Inc
89,410,Purple Heart Homes Inc


In [22]:
data = data.drop(['User ID', 'Advised Charity'], axis=1)

# Drop Rows with $0 Donation
data = data.loc[data.Amount > 0]

In [23]:
data.head()

,Amount,user_id,charity_id
0,25.0,959,439
1,1.0,963,439
2,1.0,961,439
3,20.0,593,557
4,20.0,593,557


In [24]:
# Create lists of all users, charities and amounts
users = list(np.sort(data['user_id'].unique()))
charities = list(np.sort(data['charity_id'].unique()))
amounts = list(data.Amount)

# Get the rows and columns for our new matrix
rows = data['user_id'].astype(int)
cols = data['charity_id'].astype(int)

# Contruct a sparse matrix for our users and items containing amounts
data_sparse = sparse.csr_matrix((amounts, (rows, cols)), shape=(len(users), len(charities)))

In [27]:
# Training The Model 
# (Creating User-Latent Vector and Item-Latent Vector)
user_vecs, item_vecs = implicit_als(data_sparse, iterations=20, features=20, alpha_val=40)

iteration 1 of 20
iteration 2 of 20
iteration 3 of 20
iteration 4 of 20
iteration 5 of 20
iteration 6 of 20
iteration 7 of 20
iteration 8 of 20
iteration 9 of 20
iteration 10 of 20
iteration 11 of 20
iteration 12 of 20
iteration 13 of 20
iteration 14 of 20
iteration 15 of 20
iteration 16 of 20
iteration 17 of 20
iteration 18 of 20
iteration 19 of 20
iteration 20 of 20


In [29]:
item_lookup.head()

,charity_id,Advised Charity
0,439,Room to Grow National Inc
3,557,Wildlife Conservation Society
14,396,Play Soccer to Give Corp
85,280,Lemon Bay Junior Golf Foundation Inc
89,410,Purple Heart Homes Inc


In [36]:
item_lookup

,charity_id,Advised Charity
0,439,Room to Grow National Inc
3,557,Wildlife Conservation Society
14,396,Play Soccer to Give Corp
85,280,Lemon Bay Junior Golf Foundation Inc
89,410,Purple Heart Homes Inc
...,...,...
2354,571,Zen Hospice Project Inc
2356,322,Muscular Dystrophy Association
2357,520,Treatment Advocacy Center
2358,7,Ajiri Foundation


In [39]:
item_lookup.iloc[439]

charity_id                              336
Advised Charity    Navy Seal Foundation Inc
Name: 2125, dtype: object

In [44]:
# Charity Name --> Charity_id
item_lookup['charity_id'].loc[item_lookup['Advised Charity'] == str('Room to Grow National Inc')].iloc[0]

'439'

In [38]:
# Charity_id --> Charity Name
item_lookup['Advised Charity'].loc[item_lookup.charity_id == str(439)].iloc[0]

'Room to Grow National Inc'

In [49]:
recommend_similar_charities(item_lookup,item_vec,item_name='Navy Seal Foundation Inc');

Charities Similar to: Navy Seal Foundation Inc 

                                   charities     score
0                               Lionsraw Inc  0.021511
1  Everytown For Gun Safety Support Fund Inc  0.019936
2                   Navy Seal Foundation Inc  0.019833
3                            One Mission Inc  0.019671
4                       Hungry for Music Inc  0.018806
5                     Athlife Foundation Inc  0.015616
6                       Habitat for Humanity  0.015547
7                       Exhale to Inhale Inc  0.014914
8                   Memphis Inner City Rugby  0.014672
9                       Grateful Peoples Inc  0.014529


In [30]:
#------------------------------
# FIND SIMILAR ITEMS
#------------------------------

# Let's find similar charities to __________ (Use item_lookup to locate charity_id). 
item_id = 30

# Get the item row for selected charity.
item_vec = item_vecs[item_id].T

# Calculate the similarity score between selected charity and other charities
# and select the top 10 most similar.
scores = item_vecs.dot(item_vec).toarray().reshape(1,-1)[0]
top_10 = np.argsort(scores)[::-1][:10]

charities = []
charity_scores = []

print("Charities Similar to:", item_lookup['Advised Charity'].loc[item_lookup.charity_id == str(item_id)].iloc[0],"\n")

# Get and print the actual charity names and scores
for idx in top_10:
    charities.append(item_lookup['Advised Charity'].loc[item_lookup.charity_id == str(idx)].iloc[0])
    charity_scores.append(scores[idx])

similar = pd.DataFrame({'charities': charities, 'score': charity_scores})

print (similar)

Charities Similar to: American Society For The Prevention Of Cruelty To Animals 

                                           charities     score
0  American Society For The Prevention Of Cruelty...  0.012739
1                       Dana-Farber Cancer Institute  0.009687
2                           Play Soccer to Give Corp  0.008045
3                      New York Shakespeare Festival  0.008029
4  Alzheimers Disease And Related Disorders Assoc...  0.007571
5                          The Animal Medical Center  0.007026
6               Organization For Autism Research Inc  0.006788
7                      Planned Parenthood Global Inc  0.006787
8                       Hearing Charities Of America  0.006707
9                                   Social Good Fund  0.006656


In [58]:
# Let's say we want to recommend charities for user
user_id = 20

#------------------------------
# GET ITEMS Donated to BY USER
#------------------------------

# Let's print out what the user has donated to
consumed_idx = data_sparse[user_id,:].nonzero()[1].astype(str)
consumed_items = item_lookup.loc[item_lookup.charity_id.isin(consumed_idx)]
print (consumed_items)


# Let's generate and print our recommendations
recommendations = recommend(user_id, data_sparse, user_vecs, item_vecs, item_lookup)
print("\n User-User Recommendations:")
print (recommendations)

    charity_id                  Advised Charity
358        525  Trustees For Harvard University
912        311         Memphis Inner City Rugby

 User-User Recommendations:
                              charity     score
0         Wounded Warrior Project Inc  0.941374
1                Hungry for Music Inc  0.796742
2                Bent On Learning Inc  0.686778
3                    Bent On Learning  0.649153
4              Pat Tillman Foundation  0.624113
5  Bob Woodruff Family Foundation Inc  0.577210
6                Kenya Education Fund  0.574245
7   Temple Beth El Of Northern Valley  0.517592
8                          Spirits Up  0.489190
9                                 USO  0.482612


## Testing and Scoring the Model

In [84]:
# Splitting Data into Train and Split based on Time Series
train_df = (user_and_charity_df[user_and_charity_df['Date'] < pd.to_datetime('2019-04-01 08:24:10.798807-04:00')])
test_df = (user_and_charity_df[user_and_charity_df['Date'] > pd.to_datetime('2019-04-01 08:24:10.798807-04:00')])

In [125]:
# Train the model from training data -> Default 20 iterations ALS
user_vecs, item_vecs, data_sparse, item_lookup, user_lookup = create_collab_filtering_model(train_df)

iteration 1 of 20
iteration 2 of 20
iteration 3 of 20
iteration 4 of 20
iteration 5 of 20
iteration 6 of 20
iteration 7 of 20
iteration 8 of 20
iteration 9 of 20
iteration 10 of 20
iteration 11 of 20
iteration 12 of 20
iteration 13 of 20
iteration 14 of 20
iteration 15 of 20
iteration 16 of 20
iteration 17 of 20
iteration 18 of 20
iteration 19 of 20
iteration 20 of 20


In [126]:
user_lookup

,user_id,User ID
9,589,1629
11,610,1742
12,374,1152
13,409,1216
18,663,1988
...,...,...
2354,67,373
2356,64,356
2357,54,156
2358,8,30


In [143]:
# Get user_id from User ID in user_lookup table
def get_user_id_from_table(user_lookup,User_ID):
    '''
    '''
    try:
        user_id = user_lookup[user_lookup['User ID'] == User_ID].user_id.iloc[0]
    except:
        user_id = -1
    
    return int(user_id)

In [219]:
top_3_from_train = train_df.groupby('Advised Charity')['Amount'].sum().sort_values(ascending=False).head(500)
top_3_from_train

Advised Charity
Tzahal Shalom Of N Westchester Incorporated    31471.50
Social Good Fund                               10430.52
Memphis Inner City Rugby                       10168.00
Play Soccer to Give Corp                        8321.80
Kenya Education Fund                            6852.00
                                                 ...   
Autism Speaks Inc                                  0.50
Dayton Dragons Foundation                          0.50
Trevor Day School                                  0.50
New York Womens Foundation                         0.50
Greater Newburgh Symphony Orchestra Corp           0.50
Name: Amount, Length: 456, dtype: float64

In [220]:
top_3_pop_charities_list = list(top_3_from_train.index)

In [231]:
count = 0
pop_total = 0
rec_total = 0
rec_score = 0
pop_score = 0

for i,row in test_df.iterrows():
    user_id = get_user_id_from_table(user_lookup,int(row['User ID']))
    
    
    top_3_recs_list = []
    
    if user_id == -1:
        pop_total += 1
        top_3_recs_list = top_3_pop_charities_list
        if(row['Advised Charity']) in top_3_recs_list:
            pop_score += 1
    
    if user_id != -1:
        rec_total += 1
        # Let's generate and print our recommendations
        recommendations = recommend(user_id, data_sparse, user_vecs, item_vecs, item_lookup, top_3_from_train,num_items=3)
        
        #print("\n User-User Recommendations:")
        for char in (recommendations.charity):
            top_3_recs_list.append(char)
    
    #print (row['Advised Charity'])
    
        if(row['Advised Charity']) in top_3_recs_list:
            rec_score += 1
    
print("Pop Score:", pop_score)
print("Pop Total:", pop_total)
print("Pop Accuracy %:", round((pop_score/pop_total)*100,2))    

print("\n")

print("Rec Score:", rec_score)
print("Rec Total:", rec_total)
print("Rec Accuracy %:", round((rec_score/rec_total)*100,2))

Pop Score: 152
Pop Total: 396
Pop Accuracy %: 38.38


Rec Score: 1
Rec Total: 191
Rec Accuracy %: 0.52
